# Plonk Course - Lecture 5 - 算术约束与拷贝约束

## 电路

对于下图这样的一个电路，Prover 取定隐私数据 `witness = [x1, x2, x3, x4]` 后，想对 Verifier 证明这些 `witness` 经过这样的电路运算后的值为 `out`。

![](./img/circuit.png)

在下面的运算中取 `witness = [x1, x2, x3, x4] = [1, 2, 3, 11]`，`out = 99`。我们也可以自己验证下这组值是否满足上述电路。上述电路满足下面三个约束：

- $x_1 + x_2 = x_6$
- $x_3 \cdot x_4 = x_5$
- $x_6 \cdot x_5 = out$

那么代入 `witness = [x1, x2, x3, x4] = [1, 2, 3, 11]` 可得

- $x_6 = x_1 + x_2 = 1 + 2 = 3$
- $x_5 = x_3 \cdot x_4 = 3 \cdot 11 = 33$
- $out = x_6 \cdot x_5 = 3 \cdot 33 = 99$

因此 Prover 诚实地运算了上述电路，电路输出的 `out = 99` 是正确的。如果 Prover 想要在区块链上向 Verifier 简洁地证明这件事情，并不向 Verifer 暴露 `witness`，就不能是我们这样直接带入 `witness` 进行计算了。我们用 Plonk 协议来实现这一点。

## 矩阵

Prover 和 Verifier 共识的 $Q$ 矩阵为

$$
\begin{array}{c|c|c|c|}
i & q_L & q_R & q_M & q_C & q_O  \\
\hline
0 & 0 & 0 & 0 & 99& 1 \\
1 & 0 & 0 & 1 & 0& 1 \\
2 & 1 & 1 & 0 & 0& 1 \\
3 & 0 & 0 & 1 & 0& 1 \\
\end{array}
$$

Prover 填写电路的 $W$ 矩阵 (对 Verifier 保密) 

$$
\begin{array}{c|c|c|c|}
i & w_{a,i} & w_{b,i} & w_{c,i}  \\
\hline
0 & 0 & 0 & {\color{green}out} \\
1 & {\color{red}x_6} & {\color{blue}x_5} & {\color{green}out} \\
2 & x_1 & x_2 & {\color{red}x_6} \\
3 & x_3 & x_4 & {\color{blue}x_5} \\
\end{array}
$$

表格中相同颜色的地方需要 Prover 填写相同的值，也就是拷贝约束，Prover 需要向 Verifier 证明自己正确的填写了相同的值。

代入我们上面取的值，Prover 如实填写的 $W$ 矩阵为

$$
\begin{array}{c|c|c|c|}
i & w_{a,i} & w_{b,i} & w_{c,i}  \\
\hline
0 & 0 & 0 & {\color{green}99} \\
1 & {\color{red}3} & {\color{blue}33} & {\color{green}99} \\
2 & 1 & 2 & {\color{red}3} \\
3 & 3 & 11 & {\color{blue}33} \\
\end{array}
$$

位置编码矩阵

$$
\begin{array}{c|c|c|c|}
i & id_{a,i} & id_{b,i} & id_{c,i}  \\
\hline
0 & 0 & 4 & {\color{green}8} \\
1 & {\color{red}1} & {\color{blue}5} & {\color{green}9} \\
2 & 2 & 6 & {\color{red}10} \\
3 & 3 & 7 & {\color{blue}11} \\
\end{array}
$$

对应置换矩阵

$$
\begin{array}{c|c|c|c|}
i & \sigma_{a,i} & \sigma_{b,i} & \sigma_{c,i}  \\
\hline
0 & 0 & 4 & {\color{green}9} \\
1 & {\color{red}10} & {\color{blue}11} & {\color{green}8} \\
2 & 2 & 6 & {\color{red}1} \\
3 & 3 & 7 & {\color{blue}5} \\
\end{array}
$$

需要对上述矩阵中的每一列进行多项式编码，同时也要选定我们编码的有限域。

### 有限域

在 $W$ 矩阵，其中数的范围为 0 ～ 99 ，则 $\mathbb{F}$ 的大小至少为 100，可选取有限域的大小为素数 $p = 101$，即 $\mathbb{F}_p = \mathbb{F}_{101}$，其生成元为 2 。

In [3]:
# 有限域 F_101
F_101 = GF(101)

# 获取 F_101 的生成元
g_F = F_101.multiplicative_generator()

print("F_101的生成元为：", g_F)

F_101的生成元为： 2


对 $p - 1$ 作因子分解 $p - 1 = 100 = 2^2 \cdot 5^2$，则 $\mathbb{F}_{101}$ 存在一个大小为 4 的乘法子群，其生成元 $\omega = 10$，$\mathbb{H} = \{\omega^0 = 1, \omega^1 = 10, \omega^2 = {10}^2 = 100, \omega^3 = 10^3 = 91\}$。

In [6]:
# 验证 omega = 10 是否为群 H 的生成元
omega = F_101(10)
n = 8
for i in range(0, n + 1):
    print("%d^%d: %d" %(omega, i, F_101(omega^i)), end = ", ")

10^0: 1, 10^1: 10, 10^2: 100, 10^3: 91, 10^4: 1, 10^5: 10, 10^6: 100, 10^7: 91, 10^8: 1, 

In [8]:
# H 中的元素 H = {omega^0, omega^1, omega^2, omega^3}
H = [F_101(omega^0), F_101(omega^1), F_101(omega^2), F_101(omega^3)]
print("H = ", H)

H =  [1, 10, 100, 91]


## 多项式编码

多项式编码即是对于一组值 $f = \{f_0, f_1, f_2, \cdots, f_{n - 1}\}$ 在乘法子群 $\mathbb{H} = \{\omega_0, \omega_1, \omega_2, \cdots, \omega_{n-1}\} = \{\omega, \omega^1, \omega^2, \cdots, \omega^{n-1}\}$ ( $\omega$ 为乘法群 $\mathbb{H}$ 的生成元) 上进行 Lagrange 插值。也就是得到的多项式 $f(X)$ 满足

$$
\begin{split}
f(\omega_0) & = f_0\\
f(\omega_1) & = f_1\\
f(\omega_2) & = f_2\\
& \cdots \\
f(\omega_{n-1}) & = f_{n-1}\\
\end{split}
$$

用 Lagarange 插值来计算 $f(X)$，即

$$
f(X) = \sum_{i = 0}^{n - 1} f_i \cdot L_i(X)
$$

其中

$$
L_i(x) = \prod_{i = 0, i \neq j}^3 \frac{x - \omega_j}{\omega_i - \omega_j}
$$

In [11]:
# Lagrange 插值计算
def lagrange_interpolation(GF, H, f):
    n = len(f)
    # 先计算 L_i(X) 的分母
    L_denominator = [1] * n

    for i in range(0, n):
        for j in range(0, n):
            if i != j:
                L_denominator[i] = GF((H[i] - H[j]) * L_denominator[i])

    # 声明多项式自变量 X 在 GF 中
    R.<X> = GF[]

    # 计算 L_i(X)
    L_X = [1] * n
    for i in range(0, n):
        for j in range(0, n):
            if i != j:
                L_X[i] = (X - H[j]) * L_X[i]
        L_X[i] = (GF(1) / L_denominator[i]) * L_X[i]
    
    # 计算 f(X)
    f_X = 0
    for i in range(0, n):
        f_X = f_X + f[i] * L_X[i]

    return f_X

## 预处理

Prover 与 Verifier 需要构造 $[q_L(X)]$， $[q_R(X)]$， $[q_O(X)]$， $[q_M(X)]$， $[q_C(X)]$， $[{\sigma_a}(X)]$， $[{\sigma_b}(X)]$， $[{\sigma_c}(X)]$。
这里要用到多项式来进行编码。

### 对 $Q$ 进行多项式编码
$Q$ 矩阵为

$$
\begin{array}{c|c|c|c|}
i & q_L & q_R & q_M & q_C & q_O  \\
\hline
0 & 0 & 0 & 0 & 99& 1 \\
1 & 0 & 0 & 1 & 0& 1 \\
2 & 1 & 1 & 0 & 0& 1 \\
3 & 0 & 0 & 1 & 0& 1 \\
\end{array}
$$

In [15]:
# 计算 q_L(X), q_R(X), q_M(X), q_C(X), q_O(X)
q_L = [0, 0, 1, 0]
q_R = [0, 0, 1, 0]
q_M = [0, 1, 0, 1]
q_C = [99, 0, 0, 0]
q_O = [1, 1, 1, 1]

q_L_X = lagrange_interpolation(F_101, H, q_L)
q_R_X = lagrange_interpolation(F_101, H, q_R)
q_M_X = lagrange_interpolation(F_101, H, q_M)
q_C_X = lagrange_interpolation(F_101, H, q_C)
q_O_X = lagrange_interpolation(F_101, H, q_O)

print("q_L(X) = ", q_L_X)
print("q_R(X) = ", q_R_X)
print("q_M(X) = ", q_M_X)
print("q_C(X) = ", q_C_X)
print("q_O(X) = ", q_O_X)

q_L(X) =  25*X^3 + 76*X^2 + 25*X + 76
q_R(X) =  25*X^3 + 76*X^2 + 25*X + 76
q_M(X) =  50*X^2 + 51
q_C(X) =  50*X^3 + 50*X^2 + 50*X + 50
q_O(X) =  1


### 位置向量的优化

#### 方案一：自然数

用自然数的方式来编号是比较自然的，即 $(0, 1, 2, \cdots, 11)$。

位置编码矩阵

$$
\begin{array}{c|c|c|c|}
i & id_{a,i} & id_{b,i} & id_{c,i}  \\
\hline
0 & 0 & 4 & {\color{green}8} \\
1 & {\color{red}1} & {\color{blue}5} & {\color{green}9} \\
2 & 2 & 6 & {\color{red}10} \\
3 & 3 & 7 & {\color{blue}11} \\
\end{array}
$$

对应置换矩阵

$$
\begin{array}{c|c|c|c|}
i & \sigma_{a,i} & \sigma_{b,i} & \sigma_{c,i}  \\
\hline
0 & 0 & 4 & {\color{green}9} \\
1 & {\color{red}10} & {\color{blue}11} & {\color{green}8} \\
2 & 2 & 6 & {\color{red}1} \\
3 & 3 & 7 & {\color{blue}5} \\
\end{array}
$$

In [18]:
# 方案一：计算 sigma_a(X), sigma_b(X), sigma_c(X)
sigma_a_1 = [0, 10, 2, 3]
sigma_b_1 = [4, 11, 6, 7]
sigma_c_1 = [9, 8, 1, 5]

sigma_a_1_X = lagrange_interpolation(F_101, H, sigma_a_1)
sigma_b_1_X = lagrange_interpolation(F_101, H, sigma_b_1)
sigma_c_1_X = lagrange_interpolation(F_101, H, sigma_c_1)

print("sigma_a_1(X) = ", sigma_a_1_X)
print("sigma_b_1(X) = ", sigma_b_1_X)
print("sigma_c_1(X) = ", sigma_c_1_X)

sigma_a_1(X) =  17*X^3 + 73*X^2 + 83*X + 29
sigma_b_1(X) =  60*X^3 + 99*X^2 + 40*X + 7
sigma_c_1(X) =  60*X^3 + 75*X^2 + 45*X + 31


#### 方案二：乘法子群 $\mathbb{H}$ 的陪集

在编码 $[{\sigma_a}(X)]$， $[{\sigma_b}(X)]$， $[{\sigma_c}(X)]$ 多项式时，不一定要采用自然数编号，满足互不相等的值来标记置换即可。为了使 Verifier 自己能高效计算，可以用乘法子群不同的陪集中的元素，即

$$
\begin{split}
\vec{id}_a &= (1,\omega,\omega^2,\omega^3)\\
\vec{id}_b &= (k_1,k_1\omega,k_1\omega^2,k_1\omega^3)\\
\vec{id}_c &= (k_2,k_2\omega,k_2\omega^2,k_2\omega^3)\\
\end{split}
$$

其中 $k_i$ 的取法要保证得到的三个不同的陪集，即得到的三个陪集 $\mathbb{H}$， $k_1\mathbb{H}$ 与 $k_2 \mathbb{H}$ 不能有交集，如果有交集，那么这种编号方式就有重复的两个元素了，从而无法达到我们想要用来进行置换证明的目的。这样取的目的，让我们可以直接得到 ${id_a}(X)$， ${id_a}(X)$ 与 ${id_a}(X)$ 的表达式，无需再用 Lagrange 插值进行计算，这对 Verifier 是非常友好的。

$$
{id_a}(X) = X, \quad {id_b}(X) = k_1\cdot X, \quad  {id_c}(X) = k_2\cdot X
$$

In [21]:
# 生成 k_1 和 k_2
k_1 = F_101.random_element()
k_2 = F_101.random_element()

# k_1 不能为 0， 否则 k_1H 中每个元素为0；且不能为 H 中的元素，否则生成的陪集会和 H 相同
for i in range(0, 4):
    while k_1 == 0 or k_1== H[i]:
        k_1 = F_101.random_element()
        
#  k_2 不能为 0， 否则 k_2H 中每个元素为0；且不能是陪集 H 与 k_1H 中的元素，否则生成的陪集 k_2H 会和 H 或者 k_1H 相等
for i in range(0, 4):
    k_1H = [k_1 * H[0], k_1 * H[1], k_1 * H[2], k_1 * H[3]]
    while k_2 == 0 or k_2 == H[i] or k_2 == k_1H[i]:
        k_2 = F_101.random_element()

print("k_1 = ", k_1, ", k_2 = ", k_2)

# 计算陪集 H, k_1H, k_2H 中的元素
k_1H = [k_1 * H[0], k_1 * H[1], k_1 * H[2], k_1 * H[3]]
k_2H = [k_2 * H[0], k_2 * H[1], k_2 * H[2], k_2 * H[3]]
print("H = ", H)
print("k_1H = ", k_1H)
print("k_2H = ", k_2H)

k_1 =  53 , k_2 =  3
H =  [1, 10, 100, 91]
k_1H =  [53, 25, 48, 76]
k_2H =  [3, 30, 98, 71]


##### Bad Case

$k_1$ 与 $k_2$ 不能取自同一个陪集中的元素。

In [24]:
# bad case
bad_k_1 = F_101.random_element()
bad_k_1H = [bad_k_1 * H[0], bad_k_1 * H[1], bad_k_1 * H[2], bad_k_1 * H[3]]
print("H = ", H)
print("bad_k_1H = ", bad_k_1H)

H =  [1, 10, 100, 91]
bad_k_1H =  [60, 95, 41, 6]


现在故意取 bad_k2 为 bad_k_1H 中的元素

In [27]:
bad_k2 = bad_k_1H[2]
bad_k_2H = [bad_k2 * H[0], bad_k2 * H[1], bad_k2 * H[2], bad_k2 * H[3]]
print("bad_k_2H = ", bad_k_2H)

bad_k_2H =  [41, 6, 60, 95]


可以看到 bad_k_2H 与 bad_k_1H 中的元素完全相同。

##### 关于 $k_1$ 与 $k_2$ 的选取
在论文 [PlonK: Permutations over Lagrange-bases for
Oecumenical Noninteractive arguments of
Knowledge](https://eprint.iacr.org/2019/953.pdf) 中有关于 $k_1$ 与 $k_2$ 选取的一个描述：
> $S_{ID1}(X) = X$, $S_{ID2}(X) = k1X$, $S_{ID3}(X) = k_2X$: the identity permutation applied to $\textbf{a}$, $\textbf{b}$, $\textbf{c}$. $k_1$, $k_2 \in \mathbb{F}$ are chosen such that $H$, $k_1 \cdot H$, $k_2 \cdot H$ are distinct cosets of $H$ in $\mathbb{F}^*$, and thus consist of $3n$ distinct elements. (For example, when $\omega$ is a quadratic residue in $\mathbb{F}$, take $k_1$ to be any quadratic non-residue, and $k_2$ to be a quadratic non-residue not contained in $k_1 \cdot H$.)

下面证明下上面这句话中For example 的取法的正确性，即 "For example, when $\omega$ is a quadratic residue in $\mathbb{F}$, take $k_1$ to be any quadratic non-residue, and $k_2$ to be a quadratic non-residue not contained in $k_1 \cdot H$."

在证明之前，先介绍下二次剩余的概念与性质。

**定义[[维基百科](https://zh.wikipedia.org/wiki/%E4%BA%8C%E6%AC%A1%E5%89%A9%E4%BD%99)]（二次剩余）**一个整数 $X$ 对另一个整数 $p$ 的二次剩余指 $X$ 的平方除以 $p$ 得到的余数。当存在某个 $X$ ，式子 $X^2 \equiv d (\mod p)$成立时，称“$d$ 是模 $p$ 的二次剩余”；当对任意 $X$ ， $X^2 \equiv d (\mod p)$不成立时，称“$d$ 是模 $p$ 的二次非剩余”。

其实二次剩余可以看成是对乘法群 $\mathbb{F}_p^*$ 作一个划分。意思是对于 $\mathbb{F}_p^* = \{1, 2, 3, \cdots, p-1\}$，那么对 $\mathbb{F}^*$ 中的每一个元素取平方，得到的新的值组成的集合 $H$ 是 $\mathbb{F}_p^*$ 的一个子群，且 $H$ 中的元素都是二次剩余。因为根据二次剩余的定义，$a$ 是二次剩余当且仅当存在 $x \in \mathbb{F}_p^*$ 使得 $x^2 = a$。那么对于乘法群 $\mathbb{F}_p^*$ 中的元素，每个元素要么是二次剩余，要么是二次非剩余。我们知道两个二次剩余乘起来之后还是二次剩余，而两个二次非剩余相乘之后依然还是二次非剩余，因此所有二次剩余构成的集合 $H$ 与另一个所有二次非剩余构成的集合 $gH$ （$g$ 是 $\mathbb{F}_p^*$ 中的一个二次非剩余） 对于乘法是封闭的，因此能成为群，也构成陪集。这样$\mathbb{F}_p^*$ 的一个划分就是：$\mathbb{F}_p^* = H \cup gH$。

首先对 $\mathbb{F}^*$ 通过二次剩余作一个划分，即 $\mathbb{F}^* = K \cup kK$（$k$是一个二次非剩余，$K$是$\mathbb{F}^*$ 中所有二次剩余构成的集合）。 

现在想要证明：

**对于乘法子群 $H = \{\omega^0, \omega^1, \cdots, \omega^{n-1}\}$，如果 $\omega \in K$，那么取 $\forall k_1 \in kK$，再取 $k_2 \in kK$ 且 $k_2 \notin k_1H$，那么 $H \cap k_1H \cap k_2H = \emptyset$。**

**证明**：首先 $\omega \in K$，$k_1, k_2 \in kK$，则 $H \subset K, k_1H \subset kK, k_2H \subset kK$，而 $K \cap kK = \emptyset$，因此 $H \cap k_1H = \emptyset$，$H \cap k_2H = \emptyset$。下面只需证明 $k_1H \cap k_2H = \emptyset$ 就完成证明了。

由于在选取 $k_2$ 时我们要求 $k_2 \notin k_1H$，根据陪集的性质，自然 $k_2$ 形成的陪集 $k_2H$ 与 $k_1H$ 不交，即证得 $k_1H \cap k_2H = \emptyset$。

综上，得证 $H \cap k_1H \cap k_2H = \emptyset$。

**【例子】**

下面举一些具体 $k_1$ 和 $k_2$ 的例子来进行说明。

In [34]:
# F_101 的乘法子群 F_101^*，也就是在 F_101 中去除 0
F_101_star = [0] * 100   # 这里用数组来模拟
for i in range(1,101):
    F_101_star[i - 1] = i
print("F_101 的乘法子群 F_101^* 为: ")
print(F_101_star)

F_101 的乘法子群 F_101^* 为: 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


In [36]:
# F_101^* 中的所有二次剩余集合
K_F_101 = set()
for i in range(1, 101):
    K_F_101.add(F_101(i^2)) 
print("F_101^* 中的所有二次剩余集合 K_F_101 为:")
print(K_F_101)

F_101^* 中的所有二次剩余集合 K_F_101 为:
{1, 4, 5, 6, 9, 13, 14, 16, 17, 19, 20, 21, 22, 23, 24, 25, 30, 31, 33, 36, 37, 43, 45, 47, 49, 52, 54, 56, 58, 64, 65, 68, 70, 71, 76, 77, 78, 79, 80, 81, 82, 84, 85, 87, 88, 92, 95, 96, 97, 100}


In [38]:
# F_101^* 中的所有二次非剩余集合
kK_F_101 = set()
for i in range(1, 101):
    if i not in K_F_101:
        kK_F_101.add(i)
print("F_101^* 中的所有二次非剩余集合 kK_F_101 为:")
print(kK_F_101)

F_101^* 中的所有二次非剩余集合 kK_F_101 为:
{2, 3, 7, 8, 10, 11, 12, 15, 18, 26, 27, 28, 29, 32, 34, 35, 38, 39, 40, 41, 42, 44, 46, 48, 50, 51, 53, 55, 57, 59, 60, 61, 62, 63, 66, 67, 69, 72, 73, 74, 75, 83, 86, 89, 90, 91, 93, 94, 98, 99}


**1. 选取二次剩余 $\omega$ 生成一个乘法子群**

由于我们前面生成的 $H$ 的生成元 $\omega = 10$，可以发现是在二次非剩余集合 `kK_F_101` 中的，这里为了说明前面描述的情况，从二次剩余集合 `K_F_101` 中来选取 $\omega$ 生成一个乘法子群 $H$。下面的例子中选取的 $\omega = 6 \in K$。

In [41]:
# 选取一个二次剩余的 omega = 6 来生成 F_11^* 的乘法子群 H
H_F_101 = set()
omega_H_F_101 = 6
for i in range(0, 101):
    H_F_101.add(F_101(omega_H_F_101^i))
print("由", omega_H_F_101, "生成的乘法子群 H = ", H_F_101)

由 6 生成的乘法子群 H =  {1, 65, 36, 100, 6, 14, 17, 84, 87, 95}


**2. 选取任意的二次非剩余$k_1$**

$k_1$ 在二次非剩余集合 `kK_F_101` 中选取，这里的例子选取 `k_1 = 27`。

In [44]:
k_1_F_101 = 27
print("选取k_1 = ", k_1_F_101)

k_1_H_F_101 = set()
for e in H_F_101:
    k_1_H_F_101.add(F_101(k_1_F_101 * e))
print("陪集k_1H = ", k_1_H_F_101)

选取k_1 =  27
陪集k_1H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}


**3. 选取一个二次非剩余 $k_2$，且 $k_2 \notin k_1H $**

例如我们选取 `k_2 = 11`。

In [47]:
k_2_F_101 = 11
print("选取k_2 = ", k_2_F_101)

k_2_H_F_101 = set()
for e in H_F_101:
    k_2_H_F_101.add(F_101(k_2_F_101 * e))
print("陪集k_2H = ", k_2_H_F_101)

选取k_2 =  11
陪集k_2H =  {66, 35, 8, 11, 15, 48, 53, 86, 90, 93}


从这里我们看出三个集合 $H, k_1H, k_2H$ 是不交的。即

```
H =  {1, 65, 36, 100, 6, 14, 17, 84, 87, 95}
k_1H = 27H=  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
k_2H = 11H =  {66, 35, 8, 11, 15, 48, 53, 86, 90, 93}
```

**Case 1**: 如果更进一步计算，我们取 `k_2` 为所有不在 `k_1H` 中的二次非剩余，得到下面的结果。

In [51]:
for k in kK_F_101:
    if k not in k_1_H_F_101:
        k_2_F_101 = k
        k_2_H_F_101 = set()
        for e in H_F_101:
            k_2_H_F_101.add(F_101(k_2_F_101 * e))
        print(k_2_F_101, "H = ", k_2_H_F_101)

2 H =  {2, 99, 34, 67, 72, 73, 12, 89, 28, 29}
3 H =  {98, 3, 7, 42, 18, 51, 50, 83, 59, 94}
7 H =  {98, 3, 7, 42, 50, 51, 18, 83, 59, 94}
8 H =  {66, 35, 8, 11, 15, 48, 53, 86, 90, 93}
10 H =  {32, 69, 39, 41, 10, 44, 57, 91, 60, 62}
11 H =  {66, 35, 8, 11, 15, 48, 53, 86, 90, 93}
12 H =  {2, 67, 99, 34, 72, 73, 12, 89, 28, 29}
15 H =  {66, 35, 8, 11, 15, 48, 53, 86, 90, 93}
18 H =  {98, 3, 7, 42, 18, 83, 50, 51, 59, 94}
28 H =  {2, 67, 99, 34, 72, 73, 12, 89, 28, 29}
29 H =  {34, 67, 2, 99, 72, 73, 12, 89, 28, 29}
32 H =  {32, 69, 39, 41, 10, 44, 57, 91, 60, 62}
34 H =  {2, 34, 67, 99, 72, 73, 12, 89, 28, 29}
35 H =  {66, 35, 8, 11, 15, 48, 53, 86, 90, 93}
39 H =  {32, 69, 39, 41, 10, 44, 57, 91, 60, 62}
41 H =  {32, 69, 39, 41, 10, 44, 57, 91, 60, 62}
42 H =  {98, 3, 7, 42, 50, 83, 18, 51, 59, 94}
44 H =  {32, 69, 39, 41, 10, 44, 57, 91, 60, 62}
48 H =  {66, 35, 8, 11, 15, 48, 53, 86, 90, 93}
50 H =  {98, 3, 7, 42, 50, 83, 51, 18, 59, 94}
51 H =  {98, 3, 7, 42, 18, 83, 51, 50, 59, 9

可以看到上述生成的所有陪集 `k_2H` 和 `H` 与 `k_1H` 是不交的。

```
H =  {1, 65, 36, 100, 6, 14, 17, 84, 87, 95}
k_1H = 27H=  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
```

**Case 2**: 那如果我们故意取 `k_2` 为 `k_1H` 中的元素呢？会是什么结果呢？

In [55]:
for k in k_1_H_F_101:
    k_2_F_101 = k
    k_2_H_F_101 = set()
    for e in H_F_101:
        k_2_H_F_101.add(F_101(k_2_F_101 * e))
    print(k_2_F_101, "H = ", k_2_H_F_101)

38 H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
40 H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
74 H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
75 H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
46 H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
55 H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
26 H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
27 H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
61 H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}
63 H =  {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}


可以看到 `k_2H` 和 `k_1H = {38, 40, 74, 75, 46, 55, 26, 27, 61, 63}` 完全相同。

**Case 3**: 如果我们取 `k_2` 为 `H` 中的元素会是什么结果呢？（留给大家思考几秒钟😀）

In [59]:
for k in H_F_101:
    k_2_F_101 = k
    k_2_H_F_101 = set()
    for e in H_F_101:
        k_2_H_F_101.add(F_101(k_2_F_101 * e))
    print(k_2_F_101, "H = ", k_2_H_F_101)

1 H =  {65, 1, 36, 100, 6, 14, 17, 84, 87, 95}
65 H =  {65, 1, 36, 100, 6, 14, 17, 84, 87, 95}
36 H =  {65, 1, 36, 100, 6, 14, 17, 84, 87, 95}
100 H =  {65, 1, 100, 36, 6, 14, 17, 84, 87, 95}
6 H =  {1, 65, 36, 100, 6, 14, 17, 84, 87, 95}
14 H =  {1, 65, 100, 36, 6, 14, 17, 84, 87, 95}
17 H =  {1, 65, 36, 100, 6, 14, 17, 84, 87, 95}
84 H =  {65, 1, 100, 36, 6, 14, 17, 84, 87, 95}
87 H =  {1, 65, 100, 36, 6, 14, 17, 84, 87, 95}
95 H =  {65, 1, 100, 36, 6, 14, 17, 84, 87, 95}


自然，这时 `k_2H = H = {1, 65, 36, 100, 6, 14, 17, 84, 87, 95}`

位置编码矩阵

$$
\begin{array}{c|c|c|c|}
i & id_{a,i} & id_{b,i} & id_{c,i}  \\
\hline
0 & 1 & k_1 & {\color{green}k_2} \\
1 & {\color{red}\omega} & {\color{blue}k_1\omega} & {\color{green}k_2\omega} \\
2 & \omega^2 & k_1\omega^2 & {\color{red}k_2\omega^2} \\
3 & \omega^3 & k_1\omega^3 & {\color{blue}k_2\omega^3} \\
\end{array}
$$

对应置换矩阵

$$
\begin{array}{c|c|c|c|}
i & \sigma_{a,i} & \sigma_{b,i} & \sigma_{c,i}  \\
\hline
0 & 1 & k_1 & {\color{green}k_2\omega} \\
1 & {\color{red}k_2\omega^2} & {\color{blue}k_2\omega^3} & {\color{green}k_2} \\
2 & \omega^2 & k_1\omega^2 & {\color{red}\omega} \\
3 & \omega^3 & k_1\omega^3 & {\color{blue}k_1\omega} \\
\end{array}
$$

In [63]:
# 计算 sigma_a(X), sigma_b(X), sigma_c(X)
sigma_a = [1, k_2 * omega^2, omega^2, omega^3]
sigma_b = [k_1, k_2 * omega^3, k_1 * omega^2, k_1 * omega^3]
sigma_c = [k_2 * omega, k_2, omega, k_1 * omega]

print("sigma_a = ", sigma_a)
print("sigma_b = ", sigma_b)
print("sigma_c = ", sigma_c)

sigma_a_X = lagrange_interpolation(F_101, H, sigma_a)
sigma_b_X = lagrange_interpolation(F_101, H, sigma_b)
sigma_c_X = lagrange_interpolation(F_101, H, sigma_c)

print("sigma_a(X) = ", sigma_a_X)
print("sigma_b(X) = ", sigma_b_X)
print("sigma_c(X) = ", sigma_c_X)

sigma_a =  [1, 98, 100, 91]
sigma_b =  [53, 71, 48, 76]
sigma_c =  [30, 3, 10, 25]
sigma_a(X) =  18*X^3 + 79*X^2 + 84*X + 22
sigma_b(X) =  14*X^3 + 39*X^2 + 39*X + 62
sigma_c(X) =  51*X^3 + 3*X^2 + 60*X + 17


## 第一步

Prover 针对 $W$ 表格的每一列，构造 $[w_a(X)]$， $[w_b(X)]$， $[w_c(X)]$， $\phi(X)$ 使得

$$
q_L(X)w_a(X)+q_R(X)w_b(X)+ q_M(X)w_a(X)w_b(X) - q_O(X)w_c(X)+q_C(X) + \phi(X) = 0
$$

$W$ 矩阵为

$$
\begin{array}{c|c|c|c|}
i & w_{a,i} & w_{b,i} & w_{c,i}  \\
\hline
0 & 0 & 0 & {\color{green}99} \\
1 & {\color{red}3} & {\color{blue}33} & {\color{green}99} \\
2 & 1 & 2 & {\color{red}3} \\
3 & 3 & 11 & {\color{blue}33} \\
\end{array}
$$

In [67]:
# 计算 w_a(X), w_b(X), w_c(X)
w_a = [0, 3, 1, 3]
w_b = [0, 33, 2, 11]
w_c = [99, 99, 3, 33]

w_a_X = lagrange_interpolation(F_101, H, w_a)
w_b_X = lagrange_interpolation(F_101, H, w_b)
w_c_X = lagrange_interpolation(F_101, H, w_c)

print("w_a(X) = ", w_a_X)
print("w_b(X) = ", w_b_X)
print("w_c(X) = ", w_c_X)

# phi(X)
R.<X> = F_101[]
phi_X = 0 * X

print("phi(X) = ", phi_X)

w_a(X) =  25*X^3 + 24*X^2 + 25*X + 27
w_b(X) =  4*X^3 + 40*X^2 + 96*X + 62
w_c(X) =  88*X^3 + 43*X^2 + 61*X + 8
phi(X) =  0


## 第二步

Verifier 发送随机数 $\beta$ 与 $\gamma$。

In [70]:
beta = F_101.random_element()
gamma = F_101.random_element()
print("beta = ", beta, ", gamma = ", gamma)

beta =  22 , gamma =  74


## 第三步

Prover 构造 $[z(X)]$，使得

$$
\begin{split}
L_0(X)(z(X)-1) &= 0 \\
z(\omega\cdot X)g(X) -  z(X)f(X) &=0
\end{split}
$$

### 位置编码矩阵的选择
#### 方案一：自然数

$$
\begin{array}{c|c|c|c|}
i & id_{a,i} & id_{b,i} & id_{c,i}  \\
\hline
0 & 0 & 4 & {\color{green}8} \\
1 & {\color{red}1} & {\color{blue}5} & {\color{green}9} \\
2 & 2 & 6 & {\color{red}10} \\
3 & 3 & 7 & {\color{blue}11} \\
\end{array}
$$

In [74]:
# 方案一：自然数生成的 id_1
id_a_1 = [0, 1, 2, 3]
id_b_1 = [4, 5, 6, 7]
id_c_1 = [8, 9, 10, 11]

#### 方案二：乘法子群 $\mathbb{H}$ 的陪集
$$
\begin{array}{c|c|c|c|}
i & id_{a,i} & id_{b,i} & id_{c,i}  \\
\hline
0 & 1 & k_1 & {\color{green}k_2} \\
1 & {\color{red}\omega} & {\color{blue}k_1\omega} & {\color{green}k_2\omega} \\
2 & \omega^2 & k_1\omega^2 & {\color{red}k_2\omega^2} \\
3 & \omega^3 & k_1\omega^3 & {\color{blue}k_2\omega^3} \\
\end{array}
$$

In [77]:
# 方案二：陪集生成的 id_2
id_a = [F_101(1), omega, omega^2, omega^3]
id_b = [k_1, k_1 * omega, k_1 * omega^2, k_1 * omega^3]
id_c = [k_2, k_2 * omega, k_2 * omega^2, k_2 * omega^3]

print("id_a = ", id_a)
print("id_b = ", id_b)
print("id_c = ", id_c)

# Lagrange 插值计算 id_a(X), id_b(X), id_c(X)
# 与理论结果一致, id_a(X) = X, id_b(X) = k_1 * X, id_c(X) = k_2 * X
# 实际计算时直接代上述方程
id_a_X = lagrange_interpolation(F_101, H, id_a)
id_b_X = lagrange_interpolation(F_101, H, id_b)
id_c_X = lagrange_interpolation(F_101, H, id_c)

print("id_a(X) = ", id_a_X)
print("id_b(X) = ", id_b_X)
print("id_c(X) = ", id_c_X)

id_a =  [1, 10, 100, 91]
id_b =  [53, 25, 48, 76]
id_c =  [3, 30, 98, 71]
id_a(X) =  X
id_b(X) =  53*X
id_c(X) =  3*X


下面的计算以方案二的计算为例，将下述代码中的 `sigma_a, sigma_b, sigma_c, id_a, id_b, id_c` 分别替换为`sigma_a_1, sigma_b_1, sigma_c_1, id_a_1, id_b_1, id_c_1` 即可计算对应方案一中的多项式了。

### 计算 $f(X)$ 与 $g(X)$

$f(X)$ 与 $g(X)$ 中的值分别为 $\{f_i\}$ 与 $\{g_i\}$ 为

$$
\begin{split}
f_i &= (w_{a,i}+\beta\cdot id_{a,i}+\gamma)(w_{b,i}+\beta\cdot id_{b,i}+\gamma)(w_{c,i}+\beta\cdot id_{c,i}+\gamma) \\
g_i &= (w'_{a,i}+\beta\cdot \sigma_{a,i}+\gamma)(w'_{b,i}+\beta\cdot \sigma_{b,i}+\gamma)(w'_{c,i}+\beta\cdot \sigma_{c,i}+\gamma)
\end{split}
$$

在 $g_i$ 中 $w'_{a,i} = w_{a,i}$ ，$w'_{b,i} = w_{b,i}$ ，$w'_{c,i} = w_{c,i}$，因此

$$
\begin{split}
f_i &= (w_{a,i}+\beta\cdot id_{a,i}+\gamma)(w_{b,i}+\beta\cdot id_{b,i}+\gamma)(w_{c,i}+\beta\cdot id_{c,i}+\gamma) \\
g_i &= (w_{a,i}+\beta\cdot \sigma_{a,i}+\gamma)(w_{b,i}+\beta\cdot \sigma_{b,i}+\gamma)(w_{c,i}+\beta\cdot \sigma_{c,i}+\gamma)
\end{split}
$$

那么多项式 $f(X)$ 与 $g(X)$ 满足
$$
\begin{split}
f(X)&=\Big(w_a(X)+\beta\cdot {id_a}(X)+\gamma\Big)\Big(w_b(X)+\beta\cdot {id_b}(X)+\gamma\Big)\Big(w_c(X)+\beta\cdot {id_c}(X)+\gamma\Big)\\
g(X)&=\Big(w_a(X)+\beta\cdot {\sigma_a}(X)+\gamma\Big)\Big(w_b(X)+\beta\cdot {\sigma_b}(X)+\gamma\Big)\Big(w_c(X)+\beta\cdot {\sigma_c}(X)+\gamma\Big)\\
\end{split}
$$

In [81]:
# 计算 f(X) 与 g(X)
f_X = (w_a_X + beta * X + gamma) * (w_b_X + beta * k_1 * X + gamma) * (w_c_X + beta * k_2 * X + gamma)
g_X = (w_a_X + beta * sigma_a_X + gamma) * (w_b_X + beta * sigma_b_X + gamma) * (w_c_X + beta * sigma_c_X + gamma)

print("f(X) = ", f_X)
print("g(X) = ", g_X)

f(X) =  13*X^9 + 51*X^8 + 71*X^7 + 20*X^6 + 35*X^5 + 40*X^4 + 53*X^3 + 37*X^2 + 55*X
g(X) =  98*X^9 + 81*X^8 + 13*X^7 + 82*X^6 + 38*X^5 + 90*X^4 + 47*X^3 + 94*X^2 + 80*X + 18


In [83]:
# 计算 f 与 g
f = [f_X(omega^0), f_X(omega^1), f_X(omega^2), f_X(omega^3)]
g = [g_X(omega^0), g_X(omega^1), g_X(omega^2), f_X(omega^3)]

print("f = ", f, " g = ", g)

f =  [72, 26, 22, 42]  g =  [35, 58, 89, 42]


### 计算 $z(X)$

上面计算出 $f_i$ 与 $g_i$ ，那么由递推公式

$$
z_0 = 1, \qquad z_{i+1}=z_i\cdot \frac{f_i}{g_i}\\
$$

可计算出 $z_i$。

$$
\begin{array}{|c|c|c|}
i & \omega_i \in \mathbb{H} & z_i\\
\hline
0 & \omega_0 = \omega^0 =1 & 1\\ 
1 & \omega_1 = \omega^1 =10 & 1\cdot \frac{f_0}{g_0} \\ 
2 & \omega_2 = \omega^2 =100 & \frac{f_0}{g_0}\cdot \frac{f_1}{g_1}\\
3 & \omega_3 = \omega^3 =91 & \frac{f_0f_1}{g_0g_1}\cdot \frac{f_2}{g_2} \\
4 & \omega_4 = \omega^4 =1 & \frac{f_0f_1f_2f_3}{g_0g_1g_2g_3}=1\\
\end{array}
$$

In [86]:
# 计算 z_i
z = [1, 0, 0, 0]     # 初始化
for i in range(1, 4):
    z[i] = F_101(z[i - 1] * (f[i - 1] / g[i - 1]))
print("z_i = ", z)

# 计算 z(X)
z_X = lagrange_interpolation(F_101, H, z)

print("z(X) = ", z_X)

z_i =  [1, 54, 66, 81]
z(X) =  93*X^3 + 84*X^2 + 26*X


如果我们要计算出 $z(\omega \cdot X)$ 的多项式，则依然可以用 Lagrange 插值

$$
\begin{array}{|c|c|c|}
i & \omega_i \in \mathbb{H} & z_{\omega,i}\\
\hline
0 & \omega_0 = \omega^0 =1 & z(\omega)\\ 
1 & \omega_1 = \omega^1 =10 & z(\omega^2) \\ 
2 & \omega_2 = \omega^2 =100 & z(\omega^3) \\
3 & \omega_3 = \omega^3 =91 & z(\omega^4) \\
\end{array}
$$

In [89]:
# 计算 z(omega * X)
z_omega = [z_X(omega), z_X(omega^2), z_X(omega^3), z_X(omega^4)]
print("z_omega = ", z_omega)

z_omega_X = lagrange_interpolation(F_101, H, z_omega)
print("z_omega_X = ", z_omega_X)

z_omega =  [54, 66, 81, 1]
z_omega_X =  80*X^3 + 17*X^2 + 58*X


## 第四步

Verifier 发送随机挑战数 $\alpha$。

In [92]:
alpha = F_101.random_element()
print("alpha = ", alpha)

alpha =  91


## 第五步

Prover 计算 $h(X)$，并构造商多项式 $[t(X)]$

$$
\begin{split}
h(X) = &\ q_L(X)w_a(X)+q_R(X)w_b(X)+ q_M(X)w_a(X)w_b(X) - q_O(X)w_c(X)+q_C(X) + \phi(X) \\
 & + \alpha(z(\omega X)\cdot g(X)-z(X)\cdot f(X)) + \alpha^2(L_0(X)\cdot(z(X)-1))
\end{split}
$$

其中

$$
\begin{split}
f(X)&=\Big(w_a(X)+\beta\cdot {id_a}(X)+\gamma\Big)\Big(w_b(X)+\beta\cdot {id_b}(X)+\gamma\Big)\Big(w_c(X)+\beta\cdot {id_c}(X)+\gamma\Big)\\
g(X)&=\Big(w_a(X)+\beta\cdot {\sigma_a}(X)+\gamma\Big)\Big(w_b(X)+\beta\cdot {\sigma_b}(X)+\gamma\Big)\Big(w_c(X)+\beta\cdot {\sigma_c}(X)+\gamma\Big)\\
\end{split}
$$

其中商多项式 $t(X)=\frac{h(X)}{z_H(X)}$。


### 计算 $h(X)$

In [96]:
# 计算 L_0(X)
L0 = [1, 0, 0, 0]
L0_X = lagrange_interpolation(F_101, H, L0)
print("L0(X) = ", L0_X)

# 计算 h(X)
h_X = q_L_X * w_a_X + q_R_X * w_b_X + q_M_X * w_a_X * w_b_X - q_O_X * w_c_X + q_C_X + phi_X + alpha * (z_omega_X * g_X - z_X * f_X) + alpha * alpha * (L0_X * (z_X - 1))

print("h(X) = ", h_X)

L0(X) =  76*X^3 + 76*X^2 + 76*X + 76
h(X) =  47*X^12 + 19*X^11 + 31*X^10 + 41*X^9 + 23*X^8 + 75*X^7 + 38*X^6 + 30*X^5 + 89*X^4 + 7*X^3 + 32*X^2 + 30*X + 43


### 计算 $t(X)$

因此得到的商多项式 $t(X)$ 为

$$
t(X) = \frac{h(X)}{z_H(X)}
$$

其中 $z_H(X)$ 称为 vanishing polynomial ，即

$$
z_H(X) = (X - \omega^0)(X - \omega^1)(X - \omega^2)(X - \omega^3) = X^4 - 1
$$

In [100]:
# 计算 z_H(X)
z_H_X = X^4 - 1
print("z_H(X) = ", z_H_X)

# 计算 t(X)
t_X = h_X / z_H_X

print("t(X) = ", t_X)

quotient, remainder = h_X.quo_rem(z_H_X)
print(f"Quotient: {quotient}")
print(f"Remainder: {remainder}") 

# 确保计算正确，z_H(X) 能够整除 t(X)
assert remainder == 0

z_H(X) =  X^4 + 100
t(X) =  47*X^8 + 19*X^7 + 31*X^6 + 41*X^5 + 70*X^4 + 94*X^3 + 69*X^2 + 71*X + 58
Quotient: 47*X^8 + 19*X^7 + 31*X^6 + 41*X^5 + 70*X^4 + 94*X^3 + 69*X^2 + 71*X + 58
Remainder: 0


## 第六步
Verifier 发送随机挑战数 $\zeta$，查询上述的所有 Oracle，得到
- $\bar{w}_a=w_a(\zeta)$， $\bar{w}_b=w_b(\zeta)$， $\bar{w}_c=w_c(\zeta)$
- $\bar{q}_L=q_L(\zeta)$， $\bar{q}_R=q_R(\zeta)$， $\bar{q}_M=q_M(\zeta)$，  $\bar{q}_O=q_O(\zeta)$， $\bar{q}_C=q_C(\zeta)$
- $\bar{\sigma}_a=\sigma_a(\zeta)$， $\bar{\sigma}_b=\sigma_b(\zeta)$， $\bar{\sigma}_c=\sigma_c(\zeta)$
- $\bar{z}_{(\omega\cdot\zeta)}=z(\omega\cdot\zeta)$， $\bar{z}_{(\zeta)}=z(\zeta)$
- $\bar{t}=t(\zeta)$

In [103]:
# 随机数 zeta
zeta = F_101.random_element()
print("zeta = ", zeta)

zeta =  0


### Oracle 查询

In [106]:
# 查询 w_a(X), w_b(X), w_c(X)

w_a_bar = w_a_X(zeta)
w_b_bar = w_b_X(zeta)
w_c_bar = w_c_X(zeta)
print("w_a_bar = ", w_a_bar, ", w_b_bar = ", w_b_bar, ", w_c_bar = ", w_c_bar)

w_a_bar =  27 , w_b_bar =  62 , w_c_bar =  8


In [108]:
# 查询 q_L(X), q_R(X), q_M(X), q_O(X), q_C(X)

q_L_bar = q_L_X(zeta)
q_R_bar = q_R_X(zeta)
q_M_bar = q_M_X(zeta)
q_O_bar = q_O_X(zeta)
q_C_bar = q_C_X(zeta)

print("q_L_bar = ", q_L_bar, ", q_R_bar = ", q_R_bar, "q_M_bar = ", q_M_bar, ", q_O_bar = ", q_O_bar, ", q_C_bar = ", q_C_bar)

q_L_bar =  76 , q_R_bar =  76 q_M_bar =  51 , q_O_bar =  1 , q_C_bar =  50


In [110]:
# 查询 sigma_a(X), sigma_b(X), sigma_c(X)

sigma_a_bar = sigma_a_X(zeta)
sigma_b_bar = sigma_b_X(zeta)
sigma_c_bar = sigma_c_X(zeta)

print("sigma_a_bar = ", sigma_a_bar, ", sigma_b_bar = ", sigma_b_bar, ", sigma_c_bar = ", sigma_c_bar)

sigma_a_bar =  22 , sigma_b_bar =  62 , sigma_c_bar =  17


In [112]:
# 查询 z(X), z(omega * X), t(X)

z_omega_bar = z_omega_X(zeta)
z_bar = z_X(zeta)

print("z_omega_bar = ", z_omega_bar, ", z_bar = ", z_bar)

t_bar = t_X(zeta)

print("t_bar = ", t_bar)

z_omega_bar =  0 , z_bar =  0
t_bar =  58


### Verifier 计算
Verifier 还要自行计算
- $\bar{f}_{(\zeta)} =(\bar{w}_a+\beta\cdot \zeta + \gamma) (\bar{w}_b+\beta\cdot k_1\cdot \zeta +\gamma)(\bar{w}_c+\beta\cdot k_2 \cdot \zeta +\gamma)$
- $\bar{g}_{(\zeta)}=(\bar{w}_a+\beta\cdot \bar{\sigma}_1 + \gamma) (\bar{w}_b+\beta\cdot\bar{\sigma}_2+\gamma)(\bar{w}_c+\beta\cdot\bar{\sigma}_3+\gamma)$
- $L_0(\zeta)$
- $z_H(\zeta)$
- $\phi(\zeta)$

In [115]:
# Verifier 计算 f(zeta), g(zeta)
f_v_bar = (w_a_bar + beta * zeta + gamma) * (w_b_bar + beta * k_1 * zeta + gamma) * (w_c_bar + beta * k_2 * zeta + gamma)
g_v_bar = (w_a_bar + beta * sigma_a_bar + gamma) * (w_b_bar + beta * sigma_b_bar + gamma) * (w_c_bar + beta * sigma_c_bar + gamma)

print("f_v_bar = ", f_v_bar, ", g_v_bar = ", g_v_bar)

f_v_bar =  0 , g_v_bar =  18


In [117]:
# Verifier 计算 L_0(zeta)
L0_v = [1, 0, 0, 0]
L0_v_X = lagrange_interpolation(F_101, H, L0_v)
L0_v_zeta = L0_v_X(zeta)
print("L0_v_zeta = ", L0_v_zeta)

# Verifier 计算 z_H(zeta)
z_H_v_X = X^4 - 1
z_H_v_zeta = z_H_v_X(zeta)
print("z_H_v_zeta = ", z_H_v_zeta)

# Verifier 计算 phi(zeta)
phi_v_X = 0 * X
phi_v_zeta = phi_v_X(zeta)
print("phi_v_zeta = ", phi_v_zeta)

L0_v_zeta =  76
z_H_v_zeta =  100
phi_v_zeta =  0


## 验证步

Verifier 验证下面等式是否成立，如果成立则接受，否则拒绝。
$$
\begin{split}
& \bar{q}_L\bar{w}_a+\bar{q}_R\bar{w}_b+ \bar{q}_M\bar{w}_a\bar{w}_b - \bar{q}_O\bar{w}_c+\bar{q}_C + \phi(\zeta)  \\
& \qquad \qquad + \alpha(\bar{z}_{(\omega\cdot\zeta)}\cdot \bar{g}_{(\zeta)}-\bar{z}_{(\zeta)}\cdot \bar{f}_{(\zeta)})+ \alpha^2(L_0(\zeta)\cdot(\bar{z}_{(\zeta)}-1))\overset{?}{=}\bar{t}\cdot z_H(\zeta)
\end{split}
$$

In [120]:
# 计算等式左边
LHS = F_101(q_L_bar * w_a_bar + q_R_bar * w_b_bar + q_M_bar * w_a_bar * w_b_bar - q_O_bar * w_c_bar + q_C_bar + phi_v_zeta + alpha * (z_omega_bar * g_v_bar - z_bar * f_v_bar) + alpha * alpha * (L0_v_zeta * (z_bar - 1)))
print("等式左边计算得到 LHS = ", LHS)

等式左边计算得到 LHS =  43


In [122]:
# 计算等式右边
RHS = F_101(t_bar * z_H_v_zeta)
print("等式右边计算得到 RHS = ", RHS)

等式右边计算得到 RHS =  43


In [124]:
accept = RHS == LHS
print("Verifier 是否通过验证：", accept)

Verifier 是否通过验证： True
